In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002342700958251953
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 52.33646656105054
200000 95.90536193782751
300000 98.15779289143757
400000 99.07675422664057
500000 97.38550544018644
600000 101.23805629859517
700000 98.13874443007704
800000 95.63435851983819
900000 95.67115707378734
1000000 97.02521349413064
1100000 95.9285622743476
1200000 26.791363747441707
1300000 96.89270405654737
1400000 96.34599578526685
1500000 94.5583743194284
1600000 94.39539279507167
1700000 95.60640466214949
1800000 101.0632821123691
1900000 95.66790229638657
2000000 95.69036352558501
2100000 95.68609834030671
2200000 96.97783052370954
2300000 76.3151711268941
2400000 46.3703959757285
2500000 93.57030684418457
2600000 93.79621832124069
2700000 93.813612882918
2800000 93.26331117416345
2900000 95.93267537381266
3000000 94.41976201812118
310

29700000 69.08805862691901
29800000 76.69330397690607
29900000 104.59979213008475
30000000 102.9129218150995
30100000 101.36402121557141
30200000 43.38542109690066
30300000 103.45721016659117
30400000 105.36679644795583
30500000 103.89415746374314
30600000 100.25818750508866
30700000 102.5334644895666
30800000 99.61508780188402
30900000 98.65255984261945
31000000 94.56418953421971
31100000 27.07992655863166
31200000 99.87249513201206
31300000 96.62069146378742
31400000 97.7723948252644
31500000 101.73865926804936
31600000 101.11831595989807
31700000 101.24994126719334
31800000 103.34156611255342
31900000 102.20116731626673
32000000 57.46434268969776
32100000 89.58065835288895
32200000 107.19839380415098
32300000 99.85245551136616
32400000 101.53425419262236
32500000 34.04037694077231
32600000 102.63840268473079
32700000 100.33589942523326
32800000 99.28143488846753
32900000 100.07688308395082
33000000 99.02127205015366
33100000 94.85537120941717
33200000 95.13986501957446
33300000 89.4

59900000 95.70424824702992
60000000 93.63114490544483
60100000 98.26183409252064
60200000 97.01388341359396
60300000 36.665324928509825
60400000 92.53396486217929
60500000 90.28626636714283
60600000 90.63188605576123
60700000 86.64715104366046
60800000 84.69960766437791
60900000 85.6265588443105
61000000 23.450837625958673
61100000 90.79625340521034
61200000 85.36063358257137
61300000 86.06052809214697
61400000 84.29572035921262
61500000 89.98799298656462
61600000 91.63813632448824
61700000 36.881256622914464
61800000 100.38179377192408
61900000 99.13677553672848
62000000 95.6474982992696
62100000 98.96927685642417
62200000 98.39080141456643
62300000 98.5309257870592
62400000 100.09275662578533
62500000 97.78926553274377
62600000 50.459981488336155
62700000 81.77983779379142
62800000 94.27751173476769
62900000 91.14586369572453
63000000 85.3697232506755
63100000 87.57964690964103
63200000 90.01967310653963
63300000 23.472175265229048
63400000 93.30910142046253
63500000 89.2286732718053

90400000 47.08263902050856
90500000 62.989474961698924
90600000 78.43124767079404
90700000 73.48449860853988
90800000 81.09166178175951
90900000 19.883949239861877
91000000 80.285168924245
91100000 78.17766700972025
91200000 81.2217975652223
91300000 69.6495013904375
91400000 41.1253127913707
91500000 75.4884251773207
91600000 85.9028549443039
91700000 84.06980250471516
91800000 85.71352292791669
91900000 88.04550357398097
92000000 93.55259486739911
92100000 78.29583408418202
92200000 86.17051966342041
92300000 89.90241651149971
92400000 79.53359130517961
92500000 79.15962330704703
92600000 77.00730905559648
92700000 57.472255489659524
92800000 52.166991504253524
92900000 79.36208025625018
93000000 81.13516753286123
93100000 76.69384369352164
93200000 21.24946389594715
93300000 77.69052183434263
93400000 63.35696020685193
93500000 73.31505634311354
93600000 58.20458309988974
93700000 42.13020574470679
93800000 70.41447714258831
93900000 74.11157159280708
94000000 65.59681885132036
9410

120000000 43.08399091334144
120100000 63.42268374976625
120200000 48.80472626413551
120300000 68.89906645737628
120400000 60.894433033838006
120500000 44.157316191188535
120600000 23.7734982426867
120700000 65.75584392411068
120800000 18.34101554111837
120900000 56.17817879731974
121000000 50.22544837896252
121100000 44.43982485912133
121200000 53.70979258994749
121300000 54.99378030693445
121400000 73.85498960127727
121500000 70.30612891930758
121600000 42.552564693988664
121700000 40.433008755660595
121800000 43.25241510057685
121900000 75.56983444566762
122000000 75.94332456493315
122100000 65.33088725972675
122200000 43.65728093408739
122300000 47.70353664353626
122400000 40.27831108673776
122500000 46.962038901181884
122600000 64.55797579146878
122700000 53.67799450634349
122800000 36.312575081002585
122900000 18.69415134026784
123000000 54.710174425654756
123100000 11.413555746539576
123200000 45.580271255504684
123300000 24.050826172882687
123400000 56.75840493681442
123500000 5

...

1190900000 2.7183550279721023
1191000000 3.123332342520297
1191100000 3.3396049722607435
1191200000 3.249376361739397
1191300000 3.1510565640738903
1191400000 3.650096405683097
1191500000 3.312744586374521
1191600000 4.287009502089451
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.41719655147879
1191800000 1.637889046458943
1191900000 1.1232723356896341
1192000000 1.652097848233405
1192100000 1.9882795154641963
1192200000 1.2411451799978912
1192300000 1.628940148882763
1192400000 2.093166174050828
1192500000 1.616628504122009
1192600000 1.404366171293593
1192700000 2.2119306452827523
1192800000 1.5217593045671585
1192900000 1.189623918539299
1193000000 1.5443713340351979
1193100000 1.8311665781875852
1193200000 1.1279913817148342
best so far: 0
type: [1, 4, 1, 11, 3] 132
cases of this type: 431244
1193300000 1.9648673858911296
1193400000 1.9419624917085367
1193500000 1.8478491977689366
1193600000 1.8260372101274334
best so far: 0
type: [1, 4, 1, 33, 

1204700000 2.5029115832566817
1204800000 3.273379013444424
1204900000 2.4253023781570726
1205000000 2.5699803732764885
1205100000 3.2047318458285328
1205200000 2.169500582115991
1205300000 2.817508364881943
1205400000 3.3287305628965793
1205500000 2.398578912268983
1205600000 2.8479433381521666
1205700000 3.212129801528603
1205800000 1.780348456770473
1205900000 3.4707542721618845
1206000000 3.1378050684990386
1206100000 1.951691442534791
1206200000 3.1812705914662383
1206300000 3.198377271002144
1206400000 1.830752827156994
1206500000 3.214011043411073
1206600000 2.705038697255075
1206700000 2.151010520632217
1206800000 3.2838727064856688
1206900000 2.9359276919381516
1207000000 2.5931242970689734
1207100000 2.0335826873358225
1207200000 0.6269991400354207
1207300000 0.5847588204278549
1207400000 0.5772269768208697
1207500000 0.4920094909272492
1207600000 0.5378023153860304
1207700000 0.5792177129920489
1207800000 0.5357798858203093
1207900000 0.534602019338333
1208000000 0.5459953134

1231900000 0.9072331679652299
1232000000 0.6407134705414375
1232100000 0.9013059055243928
1232200000 0.8282044636736975
1232300000 0.6804042509352763
1232400000 1.0339351777342856
1232500000 0.7364805501527952
1232600000 0.7623479184080594
1232700000 0.8655246596771679
1232800000 0.6418776650474667
1232900000 0.91908882571933
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.975714454701384
1233100000 0.5492032394446863
1233200000 0.8248685589828822
1233300000 0.9688949899286629
1233400000 0.4938974638049602
1233500000 0.8461857879747748
1233600000 0.922566811120669
1233700000 0.48105734195143646
1233800000 0.8845430820718805
1233900000 0.8028113226896424
1234000000 0.49770217237034114
1234100000 0.8818469468150801
1234200000 0.6828704486079812
1234300000 0.5721522929123243
1234400000 0.9051963349134955
1234500000 0.6840298902661801
1234600000 0.6164442197026808
1234700000 0.951626491463446
1234800000 0.6218838416634301
1234900000 0.6284162766531176
123

1253300000 0.23638338805026476
1253400000 0.1464845795667966
1253500000 0.1814500894903077
1253600000 0.16129030395944582
1253700000 0.19868379452622928
1253800000 0.15795436282772157
1253900000 0.20603201463605295
1254000000 0.1652364959293902
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.12969359007525774
1254200000 0.19541597383743523
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.1269861237250666
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.16132478299378358
1254500000 0.09859837008114326
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type: [2, 2, 11, 1, 3] 132
cases of this type: 19602
1254600000 0.15590729535610479
best so far: 0
type: [2, 2, 11, 3, 1] 132
cases of this type: 6534
best so far: 0
type: [2, 2, 33, 1, 1] 132
cases of this type: 2178
best so far: 0
type: [2, 3, 1, 1, 22] 132
cases of this type: 702768
1254700000 0.247431286751

best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best so far: 0
type: [3, 44, 1, 1, 1] 132
cases of this type: 44
best so far: 0
type: [4, 1, 1, 1, 33] 132
cases of this type: 1185921
1266100000 0.026279364976157744
1266200000 0.07021009918440381
1266300000 0.03134905880283316
1266400000 0.05011994544482894
1266500000 0